## Sandbox for marching squares

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from marching_squares import UnitCell, Lattice

In [2]:
def center_ellipse(x,y,r,c):

    draw.ellipse([x - r, y - r, x + r, y + r],fill=c)

def center_rectangle(x,y,l,w,c):
    l = l/2
    w = w/2
    draw.rectangle([x - w, y - l, x + w, y + l],fill=c)

In [3]:
image_resolution = 1080

img = Image.new('RGB', (image_resolution, image_resolution))
draw = ImageDraw.Draw(img)



def draw_cell(unit_cell, coords):
    for beam in unit_cell.beams:
        x1 = coords[beam[0]][0]
        y1 = coords[beam[0]][1]
        x2 = coords[beam[1]][0]
        y2 = coords[beam[1]][1]
        draw.line([x1, y1, x2, y2], fill="white",width=10)

unit_cell = UnitCell("BC")
scale = 200
unit_cell.x_scale = scale
unit_cell.y_scale = scale
lattice = Lattice(unit_cell, 3, 3)
i = 0
print(unit_cell.nodes)
for coords in lattice.return_all_cell_coords():
    print(coords)
    draw_cell(unit_cell, coords)
img.save('pixels.png')

[[0 2]
 [1 3]]
[[  0 400]
 [200 600]]
[[  0 600]
 [200 800]]
[[   0  800]
 [ 200 1000]]
[[200 400]
 [400 600]]
[[200 600]
 [400 800]]
[[ 200  800]
 [ 400 1000]]
[[400 400]
 [600 600]]
[[400 600]
 [600 800]]
[[ 400  800]
 [ 600 1000]]
